In [1]:
import torch
IsTPU = False
batch_size = 2
device = 'cuda'
dtype = torch.float32
speech_path = 'data/train_mp3s/'
data_path = 'data/train.csv'
num_workers = 16
clip = 1e-5
lr = 1e-7
output_path = ''
model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model_wav2vec'
background_audio = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/audio"
background_music = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/music"
add_data_path = 'dlsprint/train.csv'
add_audio = 'dlsprint/train_files/'
add_data_path2 = 'text_train'
add_audio2 = 'RESPIN/'

In [2]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
from functions import *
import torch.nn as nn
from functools import partial
from transformers.modeling_outputs import BaseModelOutput
from transformers import WhisperForConditionalGeneration
from transformers.generation.configuration_utils import GenerationConfig
from transformers import T5ForConditionalGeneration
from transformers import AutoTokenizer,Wav2Vec2Processor
from torch.optim import AdamW,Adam,SGD
from torch.nn.utils import clip_grad_value_
from torch.cuda.amp import GradScaler
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
model = torch.load(model_path + '/'+ 't5.pth').to(device)

In [4]:
paras = list(model.encoder.fc1.parameters()) + list(model.encoder.fc2.parameters())
# paras = model.parameters()
opt = AdamW(paras,lr = lr,amsgrad=True)
#opt = Adam(paras,lr = lr)
#opt = SGD(paras,lr = lr)

In [5]:
for param in model.encoder.encoder.parameters():
    param.requires_grad = False
for param in model.decoder.parameters():
    param.requires_grad = False

In [6]:
epochs = 1
verbose = 100

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
feature_extractor = Wav2Vec2Processor.from_pretrained("arijitx/wav2vec2-xls-r-300m-bengali").feature_extractor
text = pd.read_csv(data_path)
add_data = pd.read_csv(add_data_path)
# data source 2
df = pd.read_csv(add_data_path2,names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [7]:
# "https://github.com/karoldvl/ESC-50/archive/master.zip"
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    GainTransition,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
    OneOf,
    )

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=1, leave_length_unchanged=False),
        Gain(min_gain_db=-6, max_gain_db=6, p=1),
        GainTransition(min_gain_db=-6, max_gain_db=6),
        PitchShift(min_semitones=-4, max_semitones=4, p=1),
        OneOf([AddBackgroundNoise(sounds_path=background_audio, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0),\
              AddBackgroundNoise(sounds_path=background_music, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0)]),
        AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=1),
    ]
)


In [8]:
dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
                       augmentation,orig_sr=32000, target_sr=16000)
dataset2 = AudioDataset(add_data,add_audio,\
                        lambda x:x.path,augmentation,orig_sr=48000, target_sr=16000)
dataset3 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',augmentation,orig_sr=16000, target_sr=16000)
# dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
#                        orig_sr=32000, target_sr=16000)
# dataset2 = AudioDataset(add_data,add_audio,\
#                         lambda x:x.path,orig_sr=48000, target_sr=16000)
# dataset3 = AudioDataset(df,add_audio2,\
#                         lambda x:x.code.split('_')[-1] + '.wav',orig_sr=16000, target_sr=16000)                        
dataset = Add2Data(dataset1,dataset2)
dataset = Add2Data(dataset,dataset3)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, \
                            collate_fn=partial(collate_fn_pt,tokenizer=tokenizer,feature_extractor=feature_extractor,IsTrain=True,IsWhisper=False))

In [9]:
#%debug
# audio,input_ids,attention_mask = next(iter(train_loader))
# audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),torch.asarray(input_ids,dtype=torch.long,device=device),torch.asarray(attention_mask,dtype=torch.float32,device=device)
# audio.shape,input_ids.shape
# audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)

In [10]:
# audio.shape,input_ids.shape
# audio.shape[1]/5/2/2/2/2/2/2

In [11]:
# # using wav2vec as encoder
# use_amp = True
# model.train()
# np.random.seed()
# train_loss = 0
# skip = 0
# scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
# # Training #
# for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
#     audio,input_ids,attention_mask = torch.asarray(audio,device=device),\
#                                      torch.asarray(input_ids,device=device),\
#                                      torch.asarray(attention_mask,device=device)
#     with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
#         logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
#         loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
#                                                                      input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1)/torch.sum(attention_mask[:,1:]))
#     scaler.scale(loss).backward()
#     scaler.unscale_(opt)
#     clip_grad_value_(paras,clip)
#     scaler.step(opt)
#     scaler.update()
#     opt.zero_grad()
#     #print(torch.any(torch.isnan(logits)),scaler.get_scale())
#     train_loss += loss.item()
    
#     if j>0 and j%1 == 0:
#         train_loss /= (verbose-skip)
#         print(f"iterations:{j}, loss: {train_loss:.3f}")
#         train_loss = 0
#         skip = 0
#     if j>2000:
#         break
#         # if j%50 ==0:
#         #     torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

In [14]:
# using wav2vec as encoder
model.train()
np.random.seed()
train_loss = 0
skip = 0 
# Training #
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),\
                                     torch.asarray(input_ids,dtype=torch.long,device=device),\
                                     torch.asarray(attention_mask,dtype=torch.float32,device=device)
    logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
    loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
                                                                input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1))/torch.sum(attention_mask[:,1:])
    loss.backward()
    clip_grad_value_(paras,clip)
    opt.step()
    opt.zero_grad()
    train_loss += loss.item()
    if j>0 and j%verbose == 0:
        train_loss /= (verbose-skip)
        print(f"iterations:{j}, loss: {train_loss:.3f}")
        train_loss = 0
        skip = 0
    if j>10000:
        break
        # if j%50 ==0:
        #     torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

iterations:100, loss: 19.871
iterations:200, loss: 19.228
iterations:300, loss: 18.430
iterations:400, loss: 18.490
iterations:500, loss: 18.492
iterations:600, loss: 18.044
iterations:700, loss: 17.768
iterations:800, loss: 17.462
iterations:900, loss: 17.268
iterations:1000, loss: 16.940
iterations:1100, loss: 16.764
iterations:1200, loss: 16.173
iterations:1300, loss: 16.043
iterations:1400, loss: 15.794
iterations:1500, loss: 15.820
iterations:1600, loss: 15.250
iterations:1700, loss: 15.060
iterations:1800, loss: 14.910
iterations:1900, loss: 14.554
iterations:2000, loss: 14.382
iterations:2100, loss: 13.944
iterations:2200, loss: 14.031
iterations:2300, loss: 13.501
iterations:2400, loss: 13.307
iterations:2500, loss: 13.214
iterations:2600, loss: 12.987
iterations:2700, loss: 12.699
iterations:2800, loss: 12.424
iterations:2900, loss: 12.456
iterations:3000, loss: 12.084
iterations:3100, loss: 12.114
iterations:3200, loss: 11.901
iterations:3300, loss: 11.875
iterations:3400, lo

In [15]:
# using wav2vec as encoder
model.train()
np.random.seed()
train_loss = 0
skip = 0 
# Training #
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),\
                                     torch.asarray(input_ids,dtype=torch.long,device=device),\
                                     torch.asarray(attention_mask,dtype=torch.float32,device=device)
    logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
    loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
                                                                input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1))/torch.sum(attention_mask[:,1:])
    loss.backward()
    clip_grad_value_(paras,clip)
    opt.step()
    opt.zero_grad()
    train_loss += loss.item()
    if j>0 and j%verbose == 0:
        train_loss /= (verbose-skip)
        print(f"iterations:{j}, loss: {train_loss:.3f}")
        train_loss = 0
        skip = 0
    if j>10000:
        break
        # if j%50 ==0:
        #     torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

iterations:100, loss: 8.742
iterations:200, loss: 8.802
iterations:300, loss: 8.739
iterations:400, loss: 8.702
iterations:500, loss: 8.624
iterations:600, loss: 8.549
iterations:700, loss: 8.629
iterations:800, loss: 8.633
iterations:900, loss: 8.588
iterations:1000, loss: 8.604
iterations:1100, loss: 8.563
iterations:1200, loss: 8.623
iterations:1300, loss: 8.605
iterations:1400, loss: 8.571
iterations:1500, loss: 8.494
iterations:1600, loss: 8.617
iterations:1700, loss: 8.612
iterations:1800, loss: 8.577
iterations:1900, loss: 8.554
iterations:2000, loss: 8.505
iterations:2100, loss: 8.605
iterations:2200, loss: 8.437
iterations:2300, loss: 8.587
iterations:2400, loss: 8.410
iterations:2500, loss: 8.496
iterations:2600, loss: 8.455
iterations:2700, loss: 8.561
iterations:2800, loss: 8.529
iterations:2900, loss: 8.502
iterations:3000, loss: 8.390
iterations:3100, loss: 8.482
iterations:3200, loss: 8.342
iterations:3300, loss: 8.315
iterations:3400, loss: 8.352
iterations:3500, loss: 

In [16]:
# using wav2vec as encoder
verbose = 500
model.train()
np.random.seed()
train_loss = 0
skip = 0 
# Training #
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),\
                                     torch.asarray(input_ids,dtype=torch.long,device=device),\
                                     torch.asarray(attention_mask,dtype=torch.float32,device=device)
    logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
    loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
                                                                input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1))/torch.sum(attention_mask[:,1:])
    loss.backward()
    clip_grad_value_(paras,clip)
    opt.step()
    opt.zero_grad()
    train_loss += loss.item()
    if j>0 and j%verbose == 0:
        train_loss /= (verbose-skip)
        print(f"iterations:{j}, loss: {train_loss:.3f}")
        train_loss = 0
        skip = 0
    if j>10000:
        break
        # if j%50 ==0:
        #     torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

iterations:500, loss: 8.219
iterations:1000, loss: 8.291
iterations:1500, loss: 8.248
iterations:2000, loss: 8.156
iterations:2500, loss: 8.170
iterations:3000, loss: 8.132
iterations:3500, loss: 8.218
iterations:4000, loss: 8.153
iterations:4500, loss: 8.203
iterations:5000, loss: 8.190
iterations:5500, loss: 8.122
iterations:6000, loss: 8.160
iterations:6500, loss: 8.187
iterations:7000, loss: 8.132
iterations:7500, loss: 8.121
iterations:8000, loss: 8.144
iterations:8500, loss: 8.150
iterations:9000, loss: 8.076
iterations:9500, loss: 8.126
iterations:10000, loss: 8.109


In [17]:
torch.save(model, model_path + '/t5.pth')